In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessio

In [ ]:
! pip install -U sentence-transformers
! pip install langchain 
! pip install chromadb 
! pip install InstructorEmbedding
! pip install tiktoken
! pip install faiss-gpu

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer,util
import json
from langchain.document_loaders import CSVLoader,TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

from InstructorEmbedding import INSTRUCTOR
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time
from tqdm.notebook import tqdm
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR

# Loading the Dataset

In [ ]:
dataset = load_dataset("AmazonScience/mintaka",trust_remote_code=True)

In [ ]:
print(dataset["test"][0])

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

data = dataset['test']

dataset2 = MyDataset(data)

df = pd.DataFrame([sample for sample in dataset2])
df

In [ ]:
embedding = SentenceTransformerEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cuda"})

In [ ]:
OUT_PATH="/kaggle/input/triples_0_2000"

In [ ]:
def create_embeddings(question_id):
    try:
        loader = TextLoader(f"{OUT_PATH}/question_{question_id}.txt")
        documents = loader.load()
    except Exception as e:
        print(e)
        return
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    db_embeddings = FAISS.from_documents(texts,embedding)
    db_embeddings.save_local(f"/kaggle/working/embeddings/question_{question_id}")
    print("After Saving")
    

 

In [ ]:
start  = 1500
end = 2000
for i in range(start,end):
    create_embeddings(i)
    print(f"Question {i} Complete")
    
    

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()